### Create GPT-4o classifications on test data ###

In [ ]:
import os
import pandas as pd
import numpy as np

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt, MentalHealth
from llmt.llmtools import process_prompt
from llmt.openai import OpenAI, create_messages

In [ ]:
# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
test_file_name = f'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df = pd.read_parquet(test_file)
# Filter the labeled data
df = df.loc[df['dset'] == 'train']
display(df.head())
print(df.shape)

### Define the prompts and the response format ###

In [ ]:
mental_health_business_definition = Prompt().load(prompt_name='mental_health_definition')
system_prompt = Prompt().load(prompt_name='mental_health_system_prompt')

system_prompt += f' according to the following definition: {mental_health_business_definition}'

# For the user prompt, we want to include the name of the business and the description
def create_user_prompt(name: str, description: str):
    user_prompt = f"""
        The business or provider {name} is described as: {description} 
        Does the business provide mental or behavioral health care services?
        """
    return process_prompt(user_prompt)

# Load the response format
response_format = MentalHealth

In [ ]:
# Create an example for messages to be sent
idx = 10
df_idx = df.iloc[idx]
user_prompt = create_user_prompt(name=df_idx['name'], description=df_idx['description'])

messages = create_messages(system_prompt=system_prompt, user_prompt=user_prompt)
print(*messages, sep='\n\n')

In [ ]:
# Lets send this prompt to the model
client = OpenAI().create_client()
response = client.beta.chat.completions.parse(
    model='gpt-4o',
    messages=messages,
    temperature=0,
    response_format=response_format)
response_choice = response.choices[0]

In [ ]:
# Here is the reponse that we want
response_message_dict = response_choice.message.parsed.model_dump()
print(response_message_dict)

# We can get other useful information
response_dict = response_choice.model_dump()

# Create an output dictionary
output_dict = response_message_dict.copy()
output_dict.update({'refusal': response_dict.get('refusal')})